In [6]:
!pip install scipy


In [49]:
import numpy as np
from scipy.spatial.transform import Rotation as R
from scipy.ndimage.filters import laplace
from CMD import mrc_obj


In [3]:
class TBL:
    def __init__(self):
        self.t = np.array([0,0,0])
        self.r = np.array([0,0,0],dtype=float)
        self.code=0
        self.sco=0

In [44]:
def Euler2mtx(a1,b1,c1,axis="zyx"):
    r = R.from_euler(axis,[a1,b1,c1],degrees=True)
    return r.as_matrix()

def mtxmtx(a,b):
    return np.matmul(a,b)
def rot_cd(inp,m):
    return np.dot(m,inp)
def inverse_mtx(a):
    return np.linalg.inv(a)

def RotMRC(i,o,rx,ry,rz):
    


In [23]:
m = Euler2mtx(90,90,90,"xyz")
n = Euler2mtx(90,90,90,"zyx")
print("M is ",m)
print("N is",n)
print(mtxmtx(m,n))

M is  [[ 0.  0.  1.]
 [ 0.  1.  0.]
 [-1.  0.  0.]]
N is [[ 0.  0.  1.]
 [ 0. -1.  0.]
 [ 1.  0.  0.]]
[[ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]


In [43]:
a = np.array([5,2,1])
print(np.dot(m,a))


[-1.  2.  5.]
[ 1.  2. -5.]


In [48]:
def GetScore(m1,m2,T):
    px,py,pz=0
    t= [0,0,0]
    xdim = m1.xdim
    xydim=m1.xdim*m1.ydim
    xyzdim = xydim * m1.zdim
    ind1,ind2,tot,Nm,s,sco,cc_sum,pcc_sum,std1,std2,d1,d2,pstd1,pstd2,pd1,pd2,Ncc = 0
    std1 = m1.std
    std2=m2.std
    pstd1 = m1.std_norm_ave
    pstd2 = m1.std_norm_ave
    t = T
    for i in range(3):
        if(t[i]> 0.*xdim):
            t[i]-=xdim
    for x in range(xdim):
        px = x+t[0]
        for y in range(xdim):
            py = y+t[1]
            for z in range(xdim):
                pz = z + t[2]

                ind1 = xydim*z + xdim*y + x
                if(m1.dens[ind1]>0):
                    tot+=1
                
                if( px < 0 or px >= xdim):
                    continue
                if( py < 0 or py >= xdim):
                    continue
                if( pz < 0 or pz >= xdim):
                    continue

                ind2 = xydim * pz + xdim * py + px

                if(m1.dens[ind1] > 0 ):
                    d1 = m1.dens[ind1]
                    pd1 = m1.dens[ind1]-m1.ave
                else:
                    d1 = pd1 = 0
                
                if (m2.dens[ind2]>0):
                    d2 = m2.dens[ind2]
                    pd2 = m2.dens[ind2]-m2.ave
                else:
                    d2 = pd2 = 0

                cc_sum += d1*d2
                pcc_sum += pd1 * pd2

                if(m2.dens[ind2]==0):
                    continue

                if(m1.dens[ind1]==0 and m2.dens[ind2]):
                    tot+=1
                    continue
                
                s = np.dot(m1.vec[ind1],m2.vec[ind2])
                m2.sco[ind2] = s
                sco +=s
                Nm+=1

    print("Sco = ",sco)
    return sco








In [ ]:
def GetScore2(m1,m2,T,results):
    px,py,pz=0
    t= [0,0,0]
    xdim = m1.xdim
    xydim=m1.xdim*m1.ydim
    xyzdim = xydim * m1.zdim
    lap_sum = 0
    lap1,lap2 = 0
    ind1,ind2,tot,Nm,s,sco,cc_sum,pcc_sum,std1,std2,d1,d2,pstd1,pstd2,pd1,pd2,Ncc = 0
    std1 = m1.std
    std2=m2.std
    pstd1 = m1.std_norm_ave
    pstd2 = m1.std_norm_ave
    t = T
    for i in range(3):
        if(t[i]> 0.*xdim):
            t[i]-=xdim


    for x in range(xdim):
        px = x+t[0]
        for y in range(xdim):
            py = y+t[1]
            for z in range(xdim):
                pz = z + t[2]

                ind1 = xydim*z + xdim*y + x
                if(m1.dens[ind1]>0):
                    tot+=1
                
                if( px < 0 or px >= xdim):
                    continue
                if( py < 0 or py >= xdim):
                    continue
                if( pz < 0 or pz >= xdim):
                    continue

                ind2 = xydim * pz + xdim * py + px

                if(m1.dens[ind1] > 0 ):
                    d1 = m1.dens[ind1]
                    pd1 = m1.dens[ind1]-m1.ave
                else:
                    d1 = pd1 = 0
                
                if (m2.dens[ind2]>0):
                    d2 = m2.dens[ind2]
                    pd2 = m2.dens[ind2]-m2.ave
                else:
                    d2 = pd2 = 0

                cc_sum += d1*d2
                pcc_sum += pd1 * pd2
                # laplace filter
                if(d1>0 and d2>0):
                    lap1 = -6 * d1
                    lap2 = -6 * d2

                    if(x+1<xdim):
                        lap1 += m1.dens[ind1+1]
                    if(px+1<xdim):
                        lap2+= m1.dens[ind2+1]

                    if(x-1<xdim):
                        lap1 += m1.dens[ind1-1]
                    if(px-1<xdim):
                        lap2+= m1.dens[ind2-1]

                    if(y+1<xdim):
                        lap1 += m1.dens[ind1+xdim]
                    if(py+1<xdim):
                        lap2+= m1.dens[ind2+xdim]

                    if(y-1<xdim):
                        lap1 += m1.dens[ind1-xdim]
                    if(py-1<xdim):
                        lap2+= m1.dens[ind2-xdim]

                    if(z+1<xdim):
                        lap1 += m1.dens[ind1+xydim]
                    if(pz+1<xdim):
                        lap2+= m1.dens[ind2+xydim]

                    if(z-1<xdim):
                        lap1 += m1.dens[ind1-xydim]
                    if(pz-1<xdim):
                        lap2+= m1.dens[ind2-xydim]

                    lap_sum += lap1*lap2

                #filter end
                if(m2.dens[ind2]==0):
                    continue

                if(m1.dens[ind1]==0 and m2.dens[ind2]):
                    tot+=1
                    continue
                
                s = np.dot(m1.vec[ind1],m2.vec[ind2])
                m2.sco[ind2] = s
                sco +=s
                Nm+=1

    results = [sco,Nm,cc_sum/(std1*std2),pcc_sum/(pstd1*pstd2),lap_sum]
    return sco

In [58]:



laplace(mtxmtx(m,n))

array([[-2.,  0.,  0.],
       [ 0.,  4., -2.],
       [ 0., -2.,  2.]])